# Post-processing techniques

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [82]:
df = pd.read_csv('../../data/final_features_df.csv')
df.head()

,Unnamed: 0,Age,Income,faves_pca0,faves_pca1,unfaves_pca0,unfaves_pca1,accessories,alcohol,animamted,...,Drama.2,Entertainment (Variety Shows),Factual,Learning,Music,News,Religion &amp; Ethics,Sport.1,Weather,Rating_bin
0,0,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,1,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,2,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,3,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,4,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [83]:
df = df.fillna(0)

## Data preparation

In [84]:
X = df.drop(columns=['Rating_bin'])
y = df['Rating_bin']

In [85]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

## Baseline model: DecisionTree

In [132]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))
confusion_matrix(y_val, y_pred)

              precision    recall  f1-score   support

           0       0.90      0.91      0.90      7775
           1       0.40      0.39      0.39      1255

    accuracy                           0.83      9030
   macro avg       0.65      0.65      0.65      9030
weighted avg       0.83      0.83      0.83      9030



array([[7049,  726],
       [ 769,  486]])

## Métrica de fairness: Statistical Parity

In [7]:
import numpy as np

def statistical_parity(y, y_, Z, priv=None):
  if priv is None:
    values = np.unique(Z)
    counts = [np.mean(y[Z==z]) for z in values]
    priv = values[np.argmax(counts)]
    unpriv = [z for z in values if z != priv]
  else:
    unpriv = [z for z in values if z != priv]
  
  return np.array([np.mean([y_i for y_i, zi in zip(y_, Z) if zi == unp]) - np.mean([y_i for y_i, zi in zip(y_, Z) if zi == priv])
                   for unp in unpriv])

In [8]:
Z_train = X_train['Gender_M']==1
Z_val = X_val['Gender_M']==1

## Desempenho do baseline model

In [9]:
y_val_ = clf.predict(X_val)

print('F1-score:', f1_score(y_val, y_val_))
print('Statistical parity', statistical_parity(y_val, y_val_, Z_val)[0])

F1-score: 0.3940008107012566
Statistical parity -0.023685313543446607


## Estratégia 1: Threshold Optimizer

In [10]:
from fairlearn.postprocessing import ThresholdOptimizer

postprocess_est = ThresholdOptimizer(
                   estimator=clf,
                   constraints="demographic_parity",
                   objective="accuracy_score",
                   prefit=True,
                   predict_method='predict_proba')
postprocess_est.fit(X_train, y_train, sensitive_features=Z_train)

y_val_ = postprocess_est.predict(X_val, sensitive_features=Z_val)

print('F1-score:', f1_score(y_val, y_val_))
print('Statistical parity', statistical_parity(y_val, y_val_, Z_val)[0])

F1-score: 0.38884263114071604
Statistical parity -0.0029144182542162334


### Avaliando hiperparâmetros

O ThresholdOptimizer possui dois hiperparâmetros que geram impacto direto nas métricas, são eles: `constraints` e `objective`. Avaliando os resultados:

In [11]:
possible_constraints = [
    'demographic_parity',
    'selection_rate_parity',
    'equalized_odds',
    'true_positive_rate_parity',
    'true_negative_rate_parity',
    'false_positive_rate_parity',
    'false_negative_rate_parity',
]
possible_objectives = [
    'accuracy_score',
    'balanced_accuracy_score',
    'selection_rate',
    'true_positive_rate',
    'true_negative_rate',
]

impossible_pairs = [
  ['equalized_odds', 'selection_rate'],
  ['equalized_odds', 'true_positive_rate'],
  ['equalized_odds', 'true_negative_rate'],
]

results = dict()

y_val_ = clf.predict(X_val)
baseline_f1 = f1_score(y_val, y_val_)
baseline_sp = statistical_parity(y_val, y_val_, Z_val)[0]
print(f'Baseline F1: {baseline_f1}')
print(f'Baseline statistical parity: {baseline_sp}')

results['baseline']= [baseline_f1, baseline_sp]

for constraint in possible_constraints:
    for objective in possible_objectives:
        if [constraint, objective] not in impossible_pairs:
            print('---')
            print(f'Optimizing using objective "{objective}" and constraint "{constraint}"')
            postprocess_est = ThresholdOptimizer(
                   estimator=clf,
                   constraints="demographic_parity",
                   objective="true_positive_rate",
                   prefit=True,
                   predict_method='predict_proba')
            postprocess_est.fit(X_train, y_train, sensitive_features=Z_train)
            
            y_val_ = postprocess_est.predict(X_val, sensitive_features=Z_val)
            alt_f1 = f1_score(y_val, y_val_)
            alt_sp = statistical_parity(y_val, y_val_, Z_val)[0]
            print(f'F1-score: {alt_f1}')
            print(f'Statistical parity: {alt_sp}')
            results[f'{objective}__{constraint}']= [alt_f1, alt_sp]

Baseline F1: 0.3940008107012566
Baseline statistical parity: -0.023685313543446607
---
Optimizing using objective "accuracy_score" and constraint "demographic_parity"
F1-score: 0.38687258687258685
Statistical parity: -0.0025730581366494865
---
Optimizing using objective "balanced_accuracy_score" and constraint "demographic_parity"
F1-score: 0.38620689655172413
Statistical parity: 0.0003760747057938718
---
Optimizing using objective "selection_rate" and constraint "demographic_parity"
F1-score: 0.3877708978328174
Statistical parity: -0.004086678469626681
---
Optimizing using objective "true_positive_rate" and constraint "demographic_parity"
F1-score: 0.38393202008497496
Statistical parity: -0.002744702489550277
---
Optimizing using objective "true_negative_rate" and constraint "demographic_parity"
F1-score: 0.38565368299267255
Statistical parity: -0.002058125077947115
---
Optimizing using objective "accuracy_score" and constraint "selection_rate_parity"
F1-score: 0.3849144634525661
Stat

In [12]:
results

{'baseline': [0.3940008107012566, -0.023685313543446607],
 'accuracy_score__demographic_parity': [0.38687258687258685,
  -0.0025730581366494865],
 'balanced_accuracy_score__demographic_parity': [0.38620689655172413,
  0.0003760747057938718],
 'selection_rate__demographic_parity': [0.3877708978328174,
  -0.004086678469626681],
 'true_positive_rate__demographic_parity': [0.38393202008497496,
  -0.002744702489550277],
 'true_negative_rate__demographic_parity': [0.38565368299267255,
  -0.002058125077947115],
 'accuracy_score__selection_rate_parity': [0.3849144634525661,
  -0.005662656488863688],
 'balanced_accuracy_score__selection_rate_parity': [0.3866256725595696,
  -0.0005133259018400005],
 'selection_rate__selection_rate_parity': [0.3831269349845201,
  -0.004086678469626681],
 'true_positive_rate__selection_rate_parity': [0.38820826952526793,
  0.0012031176271678767],
 'true_negative_rate__selection_rate_parity': [0.38378378378378375,
  -0.003056812352221938],
 'accuracy_score__equaliz

In [13]:
best_combination = 'baseline'
best_combination_f1 = results[best_combination][0]
best_combination_sp = results[best_combination][1]

for combination, (alt_f1, alt_sp) in results.items():
    if best_combination_sp < alt_sp:
        best_combination = combination
        best_combination_f1 = alt_f1
        best_combination_sp = alt_sp

objective, constraint = best_combination.split('__')
print(f'Best objetive: {objective}')
print(f'Best constraint: {constraint}')
print(f'F1-score: {best_combination_f1}')
print(f'Statistical parity: {best_combination_sp}')

Best objetive: true_positive_rate
Best constraint: selection_rate_parity
F1-score: 0.38820826952526793
Statistical parity: 0.0012031176271678767


## Estratégia 2: Leaf Relabelling

Seguindo o que foi apresentado por [Kamiran et al.](https://www.win.tue.nl/~mpechen/publications/pubs/KamiranICDM2010.pdf)

In [113]:
import numpy as np

class Leaf:
    def __init__(self, path, node_id, u, v, w, x, transactions=None):
        self.path = path
        self.node_id = node_id
        self.acc = None
        self.disc = None
        self.ratio = None
        self.u = u
        self.v = v
        self.w = w
        self.x = x
        self.transactions = transactions

    def compute_gain(self, cnt_p, cnt_n, portion_zero, portion_one):
        n = self.u + self.w
        p = self.v + self.x
        
        if cnt_p > cnt_n:
            self.acc = n - p
            self.disc = (self.u + self.v) / portion_one - (self.w + self.x) / portion_zero

        else:
            self.acc = p - n
            self.disc = -(self.u + self.v) / portion_one + (self.w + self.x) / portion_zero

        if self.acc == 0:
            self.ratio = self.disc / -0.00000000000000000000000000000000000001
        else:
            self.ratio = self.disc / self.acc

In [129]:
def relabel_leaves(clf, x, y, y_pred, sensitive, threshold):
    for leaf in get_leaves_to_relabel(clf, x, y, y_pred, sensitive, threshold):
        if clf.tree_.value[leaf.node_id][0][0] == clf.tree_.value[leaf.node_id][0][1]:
            clf.tree_.value[leaf.node_id][0][1] += 1
        else:
            clf.tree_.value[leaf.node_id][0][0], clf.tree_.value[leaf.node_id][0][1] = \
                clf.tree_.value[leaf.node_id][0][1], clf.tree_.value[leaf.node_id][0][0]

def get_leaves_to_relabel(clf, x, y, y_pred, sensitive, threshold):
    disc_tree = discrimination(y, y_pred, sensitive)
    cnt = np.unique(sensitive, return_counts=True)[1]

    i = list()
    get_leaves_candidates(clf, x, y, sensitive, cnt, len(y), i)
    leaves = set()
    
    while rem_disc(disc_tree, leaves, threshold) > threshold and i:
        best_l = i[0]
        for leaf in i:
            if leaf.ratio > best_l.ratio:
                best_l = leaf
        leaves.add(best_l)
        i.remove(best_l)

    return leaves

def discrimination(y, y_pred, sensitive):
    w2, x2, u2, v2, b, b_not = 0, 0, 0, 0, 0, 0
    y_length = len(y)
    for index in range(0, y_length):
        if y_pred[index] == 1:
            if sensitive[index] == 0:
                if y[index] == 0:
                    w2 += 1
                elif y[index] == 1:
                    x2 += 1
            elif sensitive[index] == 1:
                if y[index] == 0:
                    u2 += 1
                elif y[index] == 1:
                    v2 += 1
        if sensitive[index] == 1:
            b += 1
        elif sensitive[index] == 0:
            b_not += 1

    w2 = w2 / y_length
    x2 = x2 / y_length
    u2 = u2 / y_length
    v2 = v2 / y_length

    b = b / y_length
    b_not = b_not / y_length

    return ((w2 + x2) / b_not) - ((u2 + v2) / b)


def get_leaves_candidates(clf, x, y, sensitive, cnt, length, leaves, node_id=0, path=tuple()):
    feature = clf.tree_.feature[node_id]
    if feature >= 0:
        tmp_path = path + ((node_id, feature, 'left'),)
        get_leaves_candidates(clf, x, y, sensitive, cnt, length, leaves,
                              clf.tree_.children_left[node_id], tmp_path)
        tmp_path = path + ((node_id, feature, 'right'),)
        get_leaves_candidates(clf, x, y, sensitive, cnt, length, leaves,
                              clf.tree_.children_right[node_id], tmp_path)
    else:
        transactions = get_transactions_by_leaf(clf, path, x)
        tmp_path = path + ((node_id, feature, 'leaf'),)

        u, v, w, x = 0, 0, 0, 0
        for transaction in transactions:
            if sensitive[transaction] == 1:
                if y[transaction] == 0:
                    u += 1
                elif y[transaction] == 1:
                    v += 1
            elif sensitive[transaction] == 0:
                if y[transaction] == 0:
                    w += 1
                elif y[transaction] == 1:
                    x += 1
        leaf = Leaf(tmp_path, node_id, u / length, v / length,
                    w / length, x / length, transactions)
        leaf.compute_gain(v + x, u + w, cnt[0] / length, cnt[1] / length)
        if leaf.disc < 0:
            leaves.append(leaf)
            
def get_transactions_by_leaf(clf, path, x):
    filtered = pd.DataFrame(x)
    for tupl in path:
        node_id = tupl[0]
        feature = tupl[1]
        if tupl[2] == 'left':
            filtered = filtered.loc[filtered[feature] <= clf.tree_.threshold[node_id]]
        elif tupl[2] == 'right':
            filtered = filtered.loc[filtered[feature] > clf.tree_.threshold[node_id]]
        else:
            raise Exception("Should not reach here")
    return list(filtered.index)

def rem_disc(disc_tree, leaves, threshold):
    s = 0
    for leaf in leaves:
        if leaf.disc < threshold:
            s += leaf.disc
    return disc_tree + s

In [127]:
sensitive = X_val['Gender_F']

y_val_np = y_val.to_numpy()
X_val_np = X_val.to_numpy()
sensitive_np = sensitive.to_numpy()

**Nota importante:** o código abaixo é bem lento, pois ele precisa percorrer toda a árvore. Portanto, ao executá-lo é esperado que ele demore cerca de 5 minutos

In [133]:
y_original_pred = clf.predict(X_val)
threshold = -0.001

relabel_leaves(clf, X_val_np, y_val_np, y_original_pred, sensitive_np, threshold)
y_pred_relabeled = clf.predict(X_val)

In [134]:
print('F1-score:', f1_score(y_val, y_pred_relabeled))
print('Statistical parity', statistical_parity(y_val, y_pred_relabeled, Z_val)[0])

F1-score: 0.4198095238095238
Statistical parity -0.00478932744985347
